In [1]:
import pandas as pd

In [3]:
import sys
sys.path.append("../") # add project dir in order to import project modules

In [4]:
from data_checkings import *

In [15]:
metadata_folder = DATA_FOLDER / "metadata"
folder = DATA_FOLDER / "tables"

let's extract some information about tables and metadata

In [16]:
info_df = exract_tables_info_from(folder, metadata_folder)

Extracting tables info: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30000/30000 [03:01<00:00, 165.16it/s]


In [17]:
info_df

,n_rows,n_cols,n_header_cols,n_cells,table_id,has_caption,footer_text,n_tables,file_id,body.content,body.headers.col,body.headers.row,footer.table_creation_date:,footer.text,header.table_id,header.text
0,1,5,4,5,Table5999962Lossadjusterchartered,1,Creation: 3Feb2013 Chad,1,0_table,"[1060, 37, 1593, 1364]",[Roberts LLC],"[Daniel Brown, Shane Barnes DDS, Nicole Carpen...",[3Feb2013],[Creation: 3Feb2013 Chad],[59.99.9.62],"[Table 59.99.9.62 Loss adjuster, chartered]"
1,4,4,3,16,Table6724Surveyorbuilding,1,Creation: 25Apr2014 Spain,1,10000_table,"[745, 748, 998, 1089, 1008, 471%, 95, 1005%, 8...","[Morton-Stephens, Brown, Roberts and Bowen, Wi...","[Megan Harper, Jessica Reid, Rebecca Winters]",[25Apr2014],[Creation: 25Apr2014 Spain],[67.24],"[Table 67.24 Surveyor, building]"
2,5,2,1,10,Table77267Healthvisitor,1,Creation: 24Aug2015 Bahrain,1,10001_table,"[1312, 542%, 1399, 77, 1254]","[Hopkins, King and Mccoy, Norris and Sons, Per...",[Christina Gray],[24Aug2015],[Creation: 24Aug2015 Bahrain],[77.26.7],[Table 77.26.7 Health visitor]
3,9,5,4,45,Table327360371941Privatemusicteacher,1,Creation: 7Mar2017 Guinea,1,10002_table,"[615, 634, 290, 409, 1391, 1143, 185%, 1261%, ...","[Myers PLC, Evans Group, Lindsey and Sons, Con...","[William Walker, Daniel Cox, Hunter Davis, Fra...",[7Mar2017],[Creation: 7Mar2017 Guinea],[32.73.60.37.19.41],[Table 32.73.60.37.19.41 Private music teacher]
4,5,2,1,10,Table5Scientistforensic,1,Creation: 12May2007 Thailand,1,10003_table,"[1534, 1588%, 992, 1047, 806]","[Barnes Group, Larson LLC, Garcia, Gonzalez an...",[Thomas Taylor],[12May2007],[Creation: 12May2007 Thailand],[5],"[Table 5 Scientist, forensic]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,2,7,6,14,Table307521207Museumgallerycurator,1,Creation: 10Sep2007 Denmark,1,9998_table,"[924, 1175, 1277, 259, 1541, 248, 211, 946%, 1...","[Chambers and Sons, Johnson-Elliott]","[Dalton Gibson, Cory Holt, Samuel Ayala, Marvi...",[10Sep2007],[Creation: 10Sep2007 Denmark],[30.75.21.20.7],[Table 30.75.21.20.7 Museum/gallery curator]
29996,1,8,7,8,Table62Scientistresearchmedical,1,Creation: 27Dec2011 Niger,1,9999_table,"[771, 1143, 780, 553, 887, 86, 864]",[Coleman and Sons],"[Kaylee Johnson, Lawrence Peters, Brianna Bent...",[27Dec2011],[Creation: 27Dec2011 Niger],[62],"[Table 62 Scientist, research (medical)]"
29997,4,8,7,32,Table7794Acupuncturist,1,Creation: 6Oct2015 Lao People's Democratic Rep...,1,999_table,"[1327, 1245, 1379, 1508, 175, 1303, 64, 257, 9...","[Griffin and Sons, Wilson, Thompson and Johnso...","[Brianna Lewis, David Kim, William Hancock, Ad...",[6Oct2015],[Creation: 6Oct2015 Lao People's Democratic Re...,[77.94],[Table 77.94 Acupuncturist]
29998,7,5,4,35,Table369050459697Riskanalyst,1,Creation: 7Feb2016 Mali,1,99_table,"[118, 1556, 1380, 1097, 214, 1118, 1460%, 669,...","[Mcdonald, Copeland and Jimenez, Welch, Vazque...","[Jacob Watkins, Donald King, Gregory Ray, Morg...",[7Feb2016],[Creation: 7Feb2016 Mali],[36.90.50.45.96.97],[Table 36.90.50.45.96.97 Risk analyst]


Question: Can we extract always caption?

In [18]:
info_df.has_caption.sum()

30000

Answer: for our specific dataset, yes, we can

Question: Is footer text (<tfoot>{text}</tfoot>) equals to header.text? 

In [20]:
info_df["footer.text"].str.len().sum()

30000

In [22]:
info_df["footer.text"] = info_df["footer.text"].map(lambda v: v[0])

In [24]:
(info_df["footer.text"] == info_df["footer_text"]).sum()

30000

Answer: for our case, we can just extract `footer.text` from `tfoot` tag as is

Question: can we extract table_id from table.id(`<table id="">`)? from which field we need to extract it?

In [25]:
(info_df["header.table_id"] == info_df["table_id"]).sum()

0

so, there is no matches, hence it's not the same    
but as it's possible to see, header.table_id is part of header.text, so need to extract it from caption

Question: from which field is possible to extract `footer.table_creation_date:`

In [28]:
if info_df["footer.table_creation_date:"].str.len().sum() == len(info_df):
    info_df["footer.table_creation_date:"] = info_df["footer.table_creation_date:"].map(lambda v: v[0])

In [30]:
sum(1 for foot_text, date_string in 
 zip(info_df["footer_text"].tolist(), info_df["footer.table_creation_date:"].tolist()) 
 if date_string in foot_text)

30000

so, all the dates are in `footer_text`

#### Summary:

* we can extract `caption` (from the `<caption>` tag)
* `footer.text` == footer text
* creation dates need to extract from footer text
* table_id is not equal to id of the html table, it's in the caption


TODO: for future, is possible to automatate such `checkings`, or `mappings` (e.g. if we have schemas, we can apply schema matching algorithms, and it seems that `similarity flooding` is promising for this case: https://dbs.uni-leipzig.de/files/research/publications/2002-1/pdf/icde2002-sf.pdf)